In [ ]:
!pip install git+https://github.com/openai/whisper.git -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:

!pip install transformers -q
!pip install torch -q
!pip install librosa -q


In [ ]:
import whisper

model = whisper.load_model("medium",device='cpu')


100%|██████████████████████████████████████| 1.42G/1.42G [00:10<00:00, 144MiB/s]


In [ ]:
from google.colab import files
uploaded = files.upload()
for filename in uploaded.keys():
    print(f"Uploaded file: {filename}")


Saving aud_1.mp3 to aud_1 (1).mp3
Uploaded file: aud_1 (1).mp3


In [ ]:
import librosa
audio_path = list(uploaded.keys())[0]
def load_audio_in_chunks(audio_path, chunk_duration=30):
    audio, sr = librosa.load(audio_path, sr=16000)
    total_duration = librosa.get_duration(y=audio, sr=sr)
    chunk_samples = int(chunk_duration * sr)
    for start_sample in range(0, len(audio), chunk_samples):
        yield audio[start_sample:start_sample + chunk_samples], sr
chunks = list(load_audio_in_chunks(audio_path))
print(f"Loaded {len(chunks)} chunks from the audio file.")



Loaded 2 chunks from the audio file.


In [ ]:
import whisper
model = whisper.load_model("medium", device="cpu")
transcriptions = []
for i, (audio_chunk, sr) in enumerate(chunks):
    result = model.transcribe(audio_chunk, fp16=False)  #FP32 is used
    transcriptions.append(result["text"])
    print(f"Transcription of chunk {i+1}/{len(chunks)}: {result['text']}")
full_transcription = " ".join(transcriptions)
print("Full Transcription:", full_transcription)



Transcription of chunk 1/2:  I'm gonna play a recording of people chanting, that is embarrassing, four times. After the fourth time, write down what you hear. Thoughts is embarrassing! Thoughts is embarrassing! Thoughts is embarrassing! Thoughts is embarrassing! Interesting, almost everyone here heard, that is embarrassing. Listen again and watch the screen. Thoughts is embarrassing! Thoughts is embarrassing! Thoughts is embarrassing!
Transcription of chunk 2/2:  Thoughts is infarct. You heard the same thing every single time. Our eyes and ears take in electrical signals which our brains interpret based on our expectations. We don't see reality. We see our reality.
Full Transcription:  I'm gonna play a recording of people chanting, that is embarrassing, four times. After the fourth time, write down what you hear. Thoughts is embarrassing! Thoughts is embarrassing! Thoughts is embarrassing! Thoughts is embarrassing! Interesting, almost everyone here heard, that is embarrassing. Listen a

In [ ]:
from transformers import MarianMTModel, MarianTokenizer
language_models = {
    "de": "Helsinki-NLP/opus-mt-en-de",  # German
    "es": "Helsinki-NLP/opus-mt-en-es",  # Spanish
    "fr": "Helsinki-NLP/opus-mt-en-fr",  # French
    "it": "Helsinki-NLP/opus-mt-en-it",  # Italian
    "zh": "Helsinki-NLP/opus-mt-en-zh",  # Chinese
    # Add more languages here as needed
}
print("Choose the target language code from the following options:")
for lang_code in language_models.keys():
    print(f"{lang_code} - {language_models[lang_code].split('-')[-1]}")
target_language = input("Enter the target language code: ")
if target_language not in language_models:
    print("Invalid language code. Please run the cell again and enter a valid code.")
else:
    model_name = language_models[target_language]
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    translation_model = MarianMTModel.from_pretrained(model_name)
    full_transcription =  " ".join(transcriptions)S
    print("Full Transcription:", full_transcription)
    inputs = tokenizer(full_transcription, return_tensors="pt", padding=True)
    translated_tokens = translation_model.generate(**inputs)
    translated_text = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
    print("Translated Text:", translated_text)



Choose the target language code from the following options:
de - de
es - es
fr - fr
it - it
zh - zh
Enter the target language code: es
Full Transcription:  I'm gonna play a recording of people chanting, that is embarrassing, four times. After the fourth time, write down what you hear. Thoughts is embarrassing! Thoughts is embarrassing! Thoughts is embarrassing! Thoughts is embarrassing! Interesting, almost everyone here heard, that is embarrassing. Listen again and watch the screen. Thoughts is embarrassing! Thoughts is embarrassing! Thoughts is embarrassing!  Thoughts is infarct. You heard the same thing every single time. Our eyes and ears take in electrical signals which our brains interpret based on our expectations. We don't see reality. We see our reality.
Translated Text: Voy a tocar una grabación de gente cantando, eso es vergonzoso, cuatro veces. Después de la cuarta vez, escribir lo que escuchas. ¡Los pensamientos es vergonzoso! ¡Los pensamientos es embarazoso! ¡Los pensamien

In [ ]:
!pip install datasets
!pip install faiss-cpu


In [ ]:
from transformers import RagTokenizer, RagRetriever, RagSequenceForGeneration
tokenizer = RagTokenizer.from_pretrained("facebook/rag-token-nq")
retriever = RagRetriever.from_pretrained("facebook/rag-token-nq", index_name="exact", use_dummy_dataset=True)
rag_model = RagSequenceForGeneration.from_pretrained("facebook/rag-token-nq", retriever=retriever)
query = translated_text
input_ids = tokenizer(query, return_tensors="pt").input_ids
generated_ids = rag_model.generate(input_ids=input_ids)
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
print("RAG Response:", response)



/usr/local/lib/python3.10/dist-packages/transformers/models/bart/configuration_bart.py:176: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in

RAG Response: [" `` law of attraction ''"]
